# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy
#jupyter nbextension enable --py gmaps

# Import API key
from api_keys import g_key
# Import API key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
# OpenWeatherMap API Key
api_key = g_key

In [23]:
api_key = weather_api_key

In [24]:
# Store filepath in a variable
# WeatherPyCities = "WeatherPy_data.csv"
#  # Read our Data file with the pandas library, including encoding
# WeatherPyCities = pd.read_csv("../Output_data/WeatherPy_data.csv, encoding="ISO-8859-1")
# WeatherPyCities.head()
WeatherPyCities = pd.read_csv("../WeatherPy/output_data/WeatherPy_data.csv", encoding="ISO-8859-1")
WeatherPyCities.head()       

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,0,Lorengau,95,PG,1612537725,80,-2.0226,147.2712,80.22,5.97
1,1,Nanortalik,0,GL,1612537832,79,60.1432,-45.2371,23.07,1.28
2,2,Weinan,0,CN,1612537832,51,34.5036,109.5089,46.99,2.24
3,3,Albany,100,US,1612537788,63,42.6001,-73.9662,35.01,4.00
4,4,Norman Wells,75,CA,1612537833,62,65.2820,-126.8329,-25.60,14.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# create variables
locations = WeatherPyCities[["Lat", "Longitude"]]
weight = WeatherPyCities["Humidity"]

In [26]:
# create custom Heatmap - customized size
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

In [28]:
# Plot Heatmap
fig = gmaps.figure(center=(40.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
holiday_cities = WeatherPyCities[(WeatherPyCities["Cloudiness"]<30) & (WeatherPyCities["Humidity"]<30) &
 (WeatherPyCities["Max Temp"]>70) & (WeatherPyCities["Max Temp"]<85) & (WeatherPyCities["Wind Speed"]<10)]
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
30,30,Bilma,0,NE,1612537837,15,18.6853,12.9164,80.91,8.34
288,288,KemisÄ,19,ET,1612537879,17,10.7167,39.8667,77.11,5.68
339,339,Bariloche,0,AR,1612537716,16,-41.1456,-71.3082,84.20,3.44
380,380,Nilanga,0,IN,1612537895,28,18.1000,76.7667,74.80,8.34
435,435,OlavarrÃ­a,0,AR,1612537905,22,-36.8927,-60.3225,84.34,5.88


In [30]:
holiday_cities.reset_index(drop=True, inplace=True)
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,30,Bilma,0,NE,1612537837,15,18.6853,12.9164,80.91,8.34
1,288,KemisÄ,19,ET,1612537879,17,10.7167,39.8667,77.11,5.68
2,339,Bariloche,0,AR,1612537716,16,-41.1456,-71.3082,84.20,3.44
3,380,Nilanga,0,IN,1612537895,28,18.1000,76.7667,74.80,8.34
4,435,OlavarrÃ­a,0,AR,1612537905,22,-36.8927,-60.3225,84.34,5.88


In [31]:
holiday_cities = holiday_cities.drop('Unnamed: 0', axis=1)
holiday_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,Bilma,0,NE,1612537837,15,18.6853,12.9164,80.91,8.34
1,KemisÄ,19,ET,1612537879,17,10.7167,39.8667,77.11,5.68
2,Bariloche,0,AR,1612537716,16,-41.1456,-71.3082,84.20,3.44
3,Nilanga,0,IN,1612537895,28,18.1000,76.7667,74.80,8.34
4,OlavarrÃ­a,0,AR,1612537905,22,-36.8927,-60.3225,84.34,5.88


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
# add new column for hotel name
hotel_df = holiday_cities
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed,Hotel Name
0,Bilma,0,NE,1612537837,15,18.6853,12.9164,80.91,8.34,
1,KemisÄ,19,ET,1612537879,17,10.7167,39.8667,77.11,5.68,
2,Bariloche,0,AR,1612537716,16,-41.1456,-71.3082,84.20,3.44,
3,Nilanga,0,IN,1612537895,28,18.1000,76.7667,74.80,8.34,
4,OlavarrÃ­a,0,AR,1612537905,22,-36.8927,-60.3225,84.34,5.88,


In [33]:
# begin looping through hotel dataframe
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"radius": 5000,
                  "types": "lodging",
                   "keyword": "Hotel",
                   "key": g_key}

        lat = row['Lat']
        long = row['Longitude']
        city = row['City']
        params['location'] = f"{lat}, {long}"

        hotel_name = requests.get(base_url, params=params).json()
        results = hotel_name['results']
        
        print(f"First hotel in {city} is {results[0]['name']}.") 
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except IndexError:
        print("No hotel found.")

No hotel found.
First hotel in KemisÄ is Osis Hotel.
First hotel in Bariloche is Águila Mora Suites & Spa.
First hotel in Nilanga is Panchdevi Apartment And Lodge.
First hotel in OlavarrÃ­a is La Josefina Hotel Boutique.


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Longitude"]]

In [35]:
# create custom Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…